Эфемериды — это таблицы положений небесных тел во времени. Для точного моделирования нужны координаты Земли, Солнца и Марса в инерциальной системе отсчета (ICRF) на моменты измерений.

Интегрируем в код SPICE (пакет программ NASA для рассчетов в области космической геометрии).


In [1]:
import spiceypy as spice
import pandas as pd
import numpy as np
from datetime import datetime

# ================== КОНФИГУРАЦИЯ ==================
KERNEL_DIR = './spice_data/'
INPUT_DATA = './output_data/mgs_doppler_ramp_CLEANED.csv'
OUTPUT_DATA = './output_data/mgs_geometry.csv'

# Словарь для преобразования station_id в названия SPICE
# ID 15, 45, 65 — это коды DSN. Их SPICE-имена: 'DSS-15', 'DSS-45', 'DSS-65'.
STATION_DICT = {
    15: 'DSS-15',  # Голдстоун
    45: 'DSS-45',  # Канберра
    65: 'DSS-65',  # Мадрид
}
# ===================================================

def load_kernels():
    """Загружает все необходимые файлы SPICE в память."""
    kernel_list = [
        KERNEL_DIR + 'naif0010.tls',              # время
        KERNEL_DIR + 'earth_assoc_itrf93.tf',     # Earth frame (ITRF93)
        KERNEL_DIR + 'earth_720101_070426.bpc',# Earth orientation (high precision)
        KERNEL_DIR + 'dsnstns.bsp',               # DSN станции
        KERNEL_DIR + 'de442s.bsp',                # планеты
        KERNEL_DIR + 'mgs_ab1.bsp',               # MGS орбита
    ]
    
    print("Загрузка ядер SPICE")
    for kernel in kernel_list:
        spice.furnsh(kernel)
        print(f"  ✓ {kernel.split('/')[-1]}")
    print("Все ядра загружены.\n")

def get_geometry_for_measurements(df):
    """
    Основная функция. Для каждого измерения получает векторы положения.
    """
    print("Начинаю расчет геометрии...")
    
    results = []
    
    for idx, row in df.iterrows():
        # 1. Подготовка времени
        utc_time = row['datetime_utc']
        # SPICE требует строку в формате: 'YYYY-MM-DD HH:MM:SS'
        if isinstance(utc_time, str):
            utc_str = utc_time
        else:  # если это datetime объект
            utc_str = utc_time.strftime('%Y-%m-%d %H:%M:%S')
        
        # Преобразуем UTC в эфимеридное время (ET) - внутреннее время SPICE
        try:
            et = spice.str2et(utc_str)
        except Exception as e:
            print(f"Ошибка времени для строки {idx}: {e}")
            continue
        
        # 2. Получаем состояние MGS относительно центра Марса
        # '-94' — это NAIF ID космического аппарата Mars Global Surveyor
        # 'J2000' — инерциальная система координат
        # 'NONE' — без коррекции за аберрацию (для начала)
        # 'MARS' — центр небесного тела, относительно которого хотим получить данные
        state_mgs, lt_mgs = spice.spkezr('-94', et, 'J2000', 'NONE', '4')
        
        # state_mgs = [X, Y, Z, VX, VY, VZ] в км и км/с
        
        # 3. Получаем положение Марса относительно Солнца (барицентра)
        pos_mars, lt_mars = spice.spkpos('4', et, 'J2000', 'NONE', 'SUN')
        # pos_mars = [X, Y, Z] в км
        
        # 4. Получаем положение наземной станции
        station_id = row['receiving_station_id']
        station_name = STATION_DICT.get(station_id)
        
        if not station_name:
            print(f"Неизвестный ID станции: {station_id} в строке {idx}")
            continue
            
        # Станция относительно центра Земли
        pos_station, lt_stat = spice.spkpos(station_name, et, 'J2000', 'NONE', 'EARTH')
        
        # 5. Получаем положение Земли относительно Солнца
        pos_earth, lt_earth = spice.spkpos('EARTH', et, 'J2000', 'NONE', 'SUN')
        
        # Собираем результат для этой строки
        result = {
            'datetime_utc': utc_str,
            'et_seconds': et,
            'station_id': station_id,
            'station_name': station_name,
            # MGS относительно Марса
            'mgs_x_km': state_mgs[0], 'mgs_y_km': state_mgs[1], 'mgs_z_km': state_mgs[2],
            'mgs_vx_km_s': state_mgs[3], 'mgs_vy_km_s': state_mgs[4], 'mgs_vz_km_s': state_mgs[5],
            # Марс относительно Солнца
            'mars_sun_x_km': pos_mars[0], 'mars_sun_y_km': pos_mars[1], 'mars_sun_z_km': pos_mars[2],
            # Станция относительно Земли
            'stat_earth_x_km': pos_station[0], 'stat_earth_y_km': pos_station[1], 'stat_earth_z_km': pos_station[2],
            # Земля относительно Солнца
            'earth_sun_x_km': pos_earth[0], 'earth_sun_y_km': pos_earth[1], 'earth_sun_z_km': pos_earth[2],
        }
        
        results.append(result)
        
        # Прогресс
        if idx % 100 == 0:
            print(f"  Обработано {idx+1} строк...")
    
    return pd.DataFrame(results)

def main():
    """Главная функция выполнения."""
    # 1. Загружаем данные
    print(f"Загрузка данных из {INPUT_DATA}...")
    df = pd.read_csv(INPUT_DATA)
    df['datetime_utc'] = pd.to_datetime(df['datetime_utc'])
    print(f"Загружено {len(df)} измерений.\n")
    
    # 2. Загружаем ядра SPICE
    load_kernels()
    
    # 3. Получаем геометрию
    geometry_df = get_geometry_for_measurements(df)
    
    # 4. Сохраняем результаты
    geometry_df.to_csv(OUTPUT_DATA, index=False)
    print(f"\n Готово! Результаты сохранены в {OUTPUT_DATA}")
    print(f"   Столбцов: {len(geometry_df.columns)}, строк: {len(geometry_df)}")
    
    # 5. Выгружаем ядра из памяти (важно!)
    spice.kclear()
    
    # 6. Показываем превью
    print("\nПервые 3 строки результата:")
    print(geometry_df[['datetime_utc', 'station_name', 'mgs_x_km', 'mgs_y_km', 'mgs_z_km']].head(3))

if __name__ == "__main__":
    main()

Загрузка данных из ./output_data/mgs_doppler_ramp_CLEANED.csv...
Загружено 1578 измерений.

Загрузка ядер SPICE
  ✓ naif0010.tls
  ✓ earth_assoc_itrf93.tf
  ✓ earth_720101_070426.bpc
  ✓ dsnstns.bsp
  ✓ de442s.bsp
  ✓ mgs_ab1.bsp
Все ядра загружены.

Начинаю расчет геометрии...
  Обработано 1 строк...
  Обработано 101 строк...
  Обработано 201 строк...
  Обработано 301 строк...
  Обработано 401 строк...
  Обработано 501 строк...
  Обработано 601 строк...
  Обработано 701 строк...
  Обработано 801 строк...
  Обработано 901 строк...
  Обработано 1001 строк...
  Обработано 1101 строк...
  Обработано 1201 строк...
  Обработано 1301 строк...
  Обработано 1401 строк...
  Обработано 1501 строк...

 Готово! Результаты сохранены в ./output_data/mgs_geometry.csv
   Столбцов: 19, строк: 1578

Первые 3 строки результата:
          datetime_utc station_name     mgs_x_km     mgs_y_km     mgs_z_km
0  1997-09-19 14:31:30       DSS-65 -5551.452977  5685.374252 -9583.035709
1  1997-09-19 14:32:30       